In [11]:
# Nạp các gói thư viện NumPy và Pandas
import pandas as pd
import numpy as np
import regex
import string

In [12]:
#Tạo đường link
import sys
import os

current = os.path.abspath('')
parent = os.path.dirname(current)
grandparent = os.path.dirname(parent)

In [13]:
#File path

## File media data
title_file = parent+"/data/processed/NLP.csv"

## File stock data
price_file = parent+'/data/processed/price07_01_2023.xlsx'

# Nap du lieu media

In [14]:
title = pd.read_csv(title_file)
title['date'] = pd.to_datetime(title['date'])
title

,date,hour,ma_ck,title,weekday,title_new
0,2023-02-24,17.5,ABC,ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán ...,Friday,miễn_nhiệm dương thị_thương kế_toán_trưởng
1,2023-02-24,17.2,ABC,"ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng...",Friday,tham_dự đại_hội_đồng cổ_đông thường_niên
2,2023-02-20,15.3,ABC,ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...,Monday,bổ_nhiệm hồ thị hà giữ chức người công_bố thôn...
3,2023-01-31,17.2,ABC,ABC: Báo cáo quản trị công ty năm 2022,Tuesday,báo_cáo quản_trị công_ty
4,2023-01-30,15.5,ACM,ACM: Đính chính Báo cáo quản trị công ty năm 2022,Monday,đính_chính báo_cáo quản_trị công_ty
...,...,...,...,...,...,...
816,2023-01-18,8.1,X77,X77: Báo cáo quản trị công ty năm 2022,Wednesday,báo_cáo quản_trị công_ty
817,2023-02-09,13.2,XPH,XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...,Thursday,bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ...
818,2023-02-09,13.2,XPH,XPH: Quyết định duy trì diện bị hạn chế giao d...,Thursday,quyết_định duy_trì diện bị hạn_chế giao_dịch h...
819,2023-02-03,8.5,XPH,XPH: Giải trình liên quan đến Báo cáo tài chín...,Friday,giải_trình liên_quan báo_cáo tài_chính niên độ


# Nap du lieu gia chung khoan

In [15]:
price_root = pd.read_excel(price_file, usecols = ['Date', 'maCK', 'Open', 'Close'])

In [16]:
price = price_root.copy()
price['Date'] = pd.DatetimeIndex(price['Date'], dayfirst=True)
price = price.rename(columns={'Date':'date',
                              'maCK':'ma_ck',
                              'Open':'open_price',
                              'Close':'close_price'})
price = price.sort_values(['ma_ck', 'date'], ascending=True)
price['open_price'] = price['open_price'].astype('str')
price['open_price'] = price['open_price'].str.replace('00.0','000')
price['open_price'] = price['open_price'].str.replace(',', '.')
# price = price[price['open_price'].isnull()==False]
price

/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/3138681378.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  price['open_price'] = price['open_price'].str.replace('00.0','000')


,date,ma_ck,open_price,close_price
30,2023-01-09,ABC,6.2,6.2
29,2023-01-10,ABC,6.2,6.2
28,2023-01-11,ABC,6.2,6.2
27,2023-01-12,ABC,6.2,6.2
26,2023-01-13,ABC,5.5,6.5
...,...,...,...,...
6791,2023-02-21,XPH,8.6,8.6
6790,2023-02-22,XPH,8.6,8.6
6789,2023-02-23,XPH,8.6,8.6
6788,2023-02-24,XPH,8.4,8.4


In [17]:
# Tam bo PVX va PXT
price['open_price'] = price['open_price'].astype('float', errors='raise')
price = price.drop_duplicates(ignore_index=True)
price

,date,ma_ck,open_price,close_price
0,2023-01-09,ABC,6.2,6.2
1,2023-01-10,ABC,6.2,6.2
2,2023-01-11,ABC,6.2,6.2
3,2023-01-12,ABC,6.2,6.2
4,2023-01-13,ABC,5.5,6.5
...,...,...,...,...
6813,2023-02-21,XPH,8.6,8.6
6814,2023-02-22,XPH,8.6,8.6
6815,2023-02-23,XPH,8.6,8.6
6816,2023-02-24,XPH,8.4,8.4


# Xu ly du lieu chung khoan

In [18]:
price['close_price_lag'] = price.groupby('ma_ck')['close_price'].shift()
price['open_price_next'] = price.groupby('ma_ck')['open_price'].shift(-1)
price = price.dropna()
price

,date,ma_ck,open_price,close_price,close_price_lag,open_price_next
1,2023-01-10,ABC,6.2,6.2,6.2,6.2
2,2023-01-11,ABC,6.2,6.2,6.2,6.2
3,2023-01-12,ABC,6.2,6.2,6.2,5.5
4,2023-01-13,ABC,5.5,6.5,6.2,6.3
5,2023-01-16,ABC,6.3,6.3,6.5,6.3
...,...,...,...,...,...,...
6812,2023-02-20,XPH,8.6,8.6,8.6,8.6
6813,2023-02-21,XPH,8.6,8.6,8.6,8.6
6814,2023-02-22,XPH,8.6,8.6,8.6,8.6
6815,2023-02-23,XPH,8.6,8.6,8.6,8.4


In [19]:
price['return_in_session'] = np.log(price['close_price']/price['open_price'])
price['return_before_session'] = np.log(price['open_price']/price['close_price_lag'])
price

/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/3476289584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['return_in_session'] = np.log(price['close_price']/price['open_price'])
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/3476289584.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['return_before_session'] = np.log(price['open_price']/price['close_price_lag'])


,date,ma_ck,open_price,close_price,close_price_lag,open_price_next,return_in_session,return_before_session
1,2023-01-10,ABC,6.2,6.2,6.2,6.2,0.000000,0.000000
2,2023-01-11,ABC,6.2,6.2,6.2,6.2,0.000000,0.000000
3,2023-01-12,ABC,6.2,6.2,6.2,5.5,0.000000,0.000000
4,2023-01-13,ABC,5.5,6.5,6.2,6.3,0.167054,-0.119801
5,2023-01-16,ABC,6.3,6.3,6.5,6.3,0.000000,-0.031253
...,...,...,...,...,...,...,...,...
6812,2023-02-20,XPH,8.6,8.6,8.6,8.6,0.000000,0.000000
6813,2023-02-21,XPH,8.6,8.6,8.6,8.6,0.000000,0.000000
6814,2023-02-22,XPH,8.6,8.6,8.6,8.6,0.000000,0.000000
6815,2023-02-23,XPH,8.6,8.6,8.6,8.4,0.000000,0.000000


In [20]:
price_stack = price[['date', 'ma_ck', 'return_in_session', 'return_before_session']].set_index(['date', 'ma_ck']).stack().reset_index()
price_stack.columns = ['date', 'ma_ck', 'session', 'return']
price_stack = price_stack.sort_values(['ma_ck', 'date', 'session'])
price_stack = price_stack[price_stack['return']!=np.inf]
price_stack = price_stack[price_stack['return']!=-np.inf]

# Ghep bang gia va NLP

In [21]:
from datetime import datetime, timedelta
title['new_date'] = title['date']
title['weekend'] = False
title['weekend'][title['weekday']=='Friday'] = title[title['weekday']=='Friday']['hour']>16
title['new_date'][title['hour']>16] = title[title['hour']>16]['new_date']+timedelta(days=1)
title['new_date'][title['weekend']==True] = title[title['weekend']==True]['new_date']+timedelta(days=2)
title['session'] = ['return_in_session' if (x>8) and (x<16) else 'return_before_session' for x in title['hour']]
title

/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/1695443992.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title['weekend'][title['weekday']=='Friday'] = title[title['weekday']=='Friday']['hour']>16
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/1695443992.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  title['new_date'][title['hour']>16] = title[title['hour']>16]['new_date']+timedelta(days=1)
/var/folders/gb/c3ht2hrn2flb60_14d0l5d_40000gn/T/ipykernel_93155/1695443992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

,date,hour,ma_ck,title,weekday,title_new,new_date,weekend,session
0,2023-02-24,17.5,ABC,ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán ...,Friday,miễn_nhiệm dương thị_thương kế_toán_trưởng,2023-02-27,True,return_before_session
1,2023-02-24,17.2,ABC,"ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng...",Friday,tham_dự đại_hội_đồng cổ_đông thường_niên,2023-02-27,True,return_before_session
2,2023-02-20,15.3,ABC,ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...,Monday,bổ_nhiệm hồ thị hà giữ chức người công_bố thôn...,2023-02-20,False,return_in_session
3,2023-01-31,17.2,ABC,ABC: Báo cáo quản trị công ty năm 2022,Tuesday,báo_cáo quản_trị công_ty,2023-02-01,False,return_before_session
4,2023-01-30,15.5,ACM,ACM: Đính chính Báo cáo quản trị công ty năm 2022,Monday,đính_chính báo_cáo quản_trị công_ty,2023-01-30,False,return_in_session
...,...,...,...,...,...,...,...,...,...
816,2023-01-18,8.1,X77,X77: Báo cáo quản trị công ty năm 2022,Wednesday,báo_cáo quản_trị công_ty,2023-01-18,False,return_in_session
817,2023-02-09,13.2,XPH,XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...,Thursday,bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ...,2023-02-09,False,return_in_session
818,2023-02-09,13.2,XPH,XPH: Quyết định duy trì diện bị hạn chế giao d...,Thursday,quyết_định duy_trì diện bị hạn_chế giao_dịch h...,2023-02-09,False,return_in_session
819,2023-02-03,8.5,XPH,XPH: Giải trình liên quan đến Báo cáo tài chín...,Friday,giải_trình liên_quan báo_cáo tài_chính niên độ,2023-02-03,False,return_in_session


In [22]:
price_stack = price_stack.rename(columns={'date':'new_date'})
df_delta = title.merge(price_stack, on=['new_date', 'ma_ck', 'session'], how='left')
df_delta

,date,hour,ma_ck,title,weekday,title_new,new_date,weekend,session,return
0,2023-02-24,17.5,ABC,ABC: Miễn nhiệm bà Dương Thị Thương - Kế toán ...,Friday,miễn_nhiệm dương thị_thương kế_toán_trưởng,2023-02-27,True,return_before_session,NaN
1,2023-02-24,17.2,ABC,"ABC: 3.3.2023, ngày GDKHQ Tham dự Đại hội đồng...",Friday,tham_dự đại_hội_đồng cổ_đông thường_niên,2023-02-27,True,return_before_session,NaN
2,2023-02-20,15.3,ABC,ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...,Monday,bổ_nhiệm hồ thị hà giữ chức người công_bố thôn...,2023-02-20,False,return_in_session,0.0
3,2023-01-31,17.2,ABC,ABC: Báo cáo quản trị công ty năm 2022,Tuesday,báo_cáo quản_trị công_ty,2023-02-01,False,return_before_session,0.0
4,2023-01-30,15.5,ACM,ACM: Đính chính Báo cáo quản trị công ty năm 2022,Monday,đính_chính báo_cáo quản_trị công_ty,2023-01-30,False,return_in_session,0.0
...,...,...,...,...,...,...,...,...,...,...
816,2023-01-18,8.1,X77,X77: Báo cáo quản trị công ty năm 2022,Wednesday,báo_cáo quản_trị công_ty,2023-01-18,False,return_in_session,0.0
817,2023-02-09,13.2,XPH,XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...,Thursday,bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ...,2023-02-09,False,return_in_session,0.0
818,2023-02-09,13.2,XPH,XPH: Quyết định duy trì diện bị hạn chế giao d...,Thursday,quyết_định duy_trì diện bị hạn_chế giao_dịch h...,2023-02-09,False,return_in_session,0.0
819,2023-02-03,8.5,XPH,XPH: Giải trình liên quan đến Báo cáo tài chín...,Friday,giải_trình liên_quan báo_cáo tài_chính niên độ,2023-02-03,False,return_in_session,0.0


In [23]:
df_delta = df_delta.drop_duplicates(ignore_index=True)
df_delta = df_delta.dropna().reset_index(drop=True)
df_delta                                    

,date,hour,ma_ck,title,weekday,title_new,new_date,weekend,session,return
0,2023-02-20,15.3,ABC,ABC: Bổ nhiệm bà Hồ Thị Hà giữ chức người công...,Monday,bổ_nhiệm hồ thị hà giữ chức người công_bố thôn...,2023-02-20,False,return_in_session,0.00000
1,2023-01-31,17.2,ABC,ABC: Báo cáo quản trị công ty năm 2022,Tuesday,báo_cáo quản_trị công_ty,2023-02-01,False,return_before_session,0.00000
2,2023-01-30,15.5,ACM,ACM: Đính chính Báo cáo quản trị công ty năm 2022,Monday,đính_chính báo_cáo quản_trị công_ty,2023-01-30,False,return_in_session,0.00000
3,2023-02-17,15.0,AGF,"AGF: 14.3.2023, ngày GDKHQ Tham dự đại hội đồn...",Friday,tham_dự đại_hội_đồng cổ_đông thường_niên,2023-02-17,False,return_in_session,0.03774
4,2023-02-13,16.5,AGF,AGF: Nghị quyết HĐQT thống nhất tổ chức ĐHĐCĐ ...,Monday,nghị_quyết hđqt thống_nhất tổ_chức đhđcđ,2023-02-14,False,return_before_session,0.00000
...,...,...,...,...,...,...,...,...,...,...
698,2023-01-18,8.1,X77,X77: Báo cáo quản trị công ty năm 2022,Wednesday,báo_cáo quản_trị công_ty,2023-01-18,False,return_in_session,0.00000
699,2023-02-09,13.2,XPH,XPH: Bổ nhiệm ông Lê Mạnh Cường giữ chức Phụ t...,Thursday,bổ_nhiệm lê mạnh cường giữ chức phụ_trách_nhiệ...,2023-02-09,False,return_in_session,0.00000
700,2023-02-09,13.2,XPH,XPH: Quyết định duy trì diện bị hạn chế giao d...,Thursday,quyết_định duy_trì diện bị hạn_chế giao_dịch h...,2023-02-09,False,return_in_session,0.00000
701,2023-02-03,8.5,XPH,XPH: Giải trình liên quan đến Báo cáo tài chín...,Friday,giải_trình liên_quan báo_cáo tài_chính niên độ,2023-02-03,False,return_in_session,0.00000


In [24]:
df_delta_sub = df_delta
df_delta_sub['title_new'] = df_delta_sub['title_new']+' '
df_final = df_delta_sub[['ma_ck', 'title_new', 'new_date', 'session', 'return']].groupby(['new_date', 'ma_ck', 'session', 'return']).sum().reset_index()
df_final['title_new'] = df_final['title_new'].str[:-1]
df_final

,new_date,ma_ck,session,return,title_new
0,2023-01-10,BHT,return_in_session,0.000000,tổ_chức đại_hội_đồng cổ_đông thường_niên
1,2023-01-10,HAG,return_before_session,-0.008830,hoàng gia lai hagl lãi trở_lại sau đức hào_phó...
2,2023-01-10,PET,return_in_session,-0.007624,thực_hiện quyền nhận cổ_tức cổ_phiếu
3,2023-01-10,PLX,return_in_session,0.048654,sau khi hạ chỉ tiêu bất_ngờ báo lãi thuế gần
4,2023-01-10,PRT,return_in_session,-0.017544,thông_báo dự_kiến tổ_chức đại_hội_đồng cổ_đông...
...,...,...,...,...,...
538,2023-02-24,SP2,return_in_session,0.000000,bổ_nhiệm bùi quế_lâm giữ chức tgđ
539,2023-02-24,STT,return_in_session,-0.111226,ký_kết hợp_đồng
540,2023-02-24,UDC,return_before_session,0.018562,nghị_quyết hđqt thông_qua phương_án sử_dụng tà...
541,2023-02-24,VOS,return_in_session,-0.004197,tổ_chức đhđcđ thường_niên


In [26]:
#Save file
df_final.to_csv(parent+'/data/processed/input_table.csv', index=False)